<a href="https://colab.research.google.com/github/syeong1218/RL/blob/master/Ch_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4장 A2C

## 4.1 배경
REINFORCE 알고리즘 정책의 두 가진 단점은 에피소드가 끝날 때까지 기다려야하는 점과 그래디언트의 분산이 매우 크다는 것이다.  
이 두 가지 단점을 개선한 알고리즘이 어드밴티지 액터-크리틱(advantage actor-critic, A2C)이다

## 4.2 그래디언트의 재구성  
* 목적함수의 그래디언트  
$$\nabla_\theta J(\theta) = E_{\tau \sim p_\theta}[\sum _{t=0} ^T (\gamma^t \nabla_\theta \log \pi_\theta(u_t|x_t)(\sum _{k=t} ^T \gamma^{k-t}r(x_k, u_k)))]$$
$$=\sum _{t=0} ^T \{E_{\tau \sim p_\theta}[\gamma^t \nabla_\theta \log  \pi_\theta(u_t|x_t)(\sum _{k=t} ^T \gamma^{k-t}r(x_k, u_k))]\}$$

$(\sum _{k=t} ^T \gamma^{k-t}r(x_k, u_k))$항은 시간스텝 k=t부터 에피소드가 종료될 때까지 받을 수 있는 감가된 예정 보상(reward-to-go)의 총합인 반환값 $G_t$이다. 이 부분을 더 명확하게 규명하는 것이 목적이다.  
궤적 $\tau$를 시간스텝 t를 기준으로 다음과 같이 두 개의 영역으로 분할한다.  
$$\tau = (x_0, u_0, \cdots, x_t, u_t) \cup (x_{t+1}, u_{t+1},\cdots,x_T,u_T)$$
$$=\tau_{x_0 : u_t}\cup\tau_{x_{t+1}:u_T}$$

이를 바탕으로 목적함수의 그래디언트는 다음과 같이 표현할 수 있다. 
$$\nabla_\theta J(\theta)=\sum _{t=0} ^T \int _{\tau_{x_0 : u_t}} \int _{\tau_{x_{t+1}:u_T}} (\gamma^t \nabla_\theta \log \pi_\theta(u_t|x_t)(\sum _{k=t} ^T \gamma^{k-t}r(x_k, u_k))) p_\theta (\tau_{x_0 : u_t}, \tau_{x_{t+1}:u_T})d\tau_{x_{t+1}:u_T} d\tau_{x_0 : u_t}$$
$$=\sum _{t=0} ^T \int _{\tau_{x_0 : u_t}} \int _{\tau_{x_{t+1}:u_T}} (\gamma^t \nabla_\theta \log \pi_\theta(u_t|x_t)(\sum _{k=t} ^T \gamma^{k-t}r(x_k, u_k))) p_\theta (\tau_{x_{t+1}:u_T} | \tau_{x_0 : u_t})p_\theta(\tau_{x_0 : u_t}) d\tau_{x_{t+1}:u_T} d\tau_{x_0 : u_t} $$
$$=\sum _{t=0} ^T \int _{\tau_{x_0 : u_t}} (\gamma^t \nabla_\theta \log \pi_\theta(u_t|x_t)[\int _{\tau_{x_{t+1}:u_T}}(\sum _{k=t} ^T \gamma^{k-t}r(x_k, u_k)) p_\theta (\tau_{x_{t+1}:u_T} | \tau_{x_0 : u_t}) d\tau_{x_{t+1}:u_T}] p_\theta(\tau_{x_0 : u_t}) d\tau_{x_0 : u_t} $$
$$=\sum _{t=0} ^T (\int _{(x_t, u_t)} \gamma^t \nabla_\theta \log \pi_\theta(u_t|x_t)Q^{\pi_\theta}(x_t, u_t)p_\theta(x_t,u_t)dx_tdu_t)$$
$$=\sum _{t=0} ^T (E_{x_t \sim d_\theta (x_t),u_t \sim \pi_\theta(u_t|x_t)}[\gamma^t \nabla_\theta \log \pi_\theta(u_t|x_t)Q^{\pi_\theta}(x_t, u_t)])$$

베이즈 정리, 마르코프 가정을 적절히 적용하여 식을 변형하였다. $\gamma^t$는 에피소드의 후반부 궤적에 있는 데이터의 이용도를 크게 떨어뜨리는 단점이 있기 때문에 제거하였다.  

원래 의 그래디언트식 $\sum _{t=0} ^T \{E_{\tau \sim p_\theta}[\gamma^t \nabla_\theta \log  \pi_\theta(u_t|x_t)(G_t)]\}$과 비교하면 반환값 $G_t$ 대신에 행동가치 함수가 쓰여진 것을 알 수 있다. 행공가치는 상태변수 $x_t$에서 행동 $u_t$를 선택하고 그러부터 정책 $\pi$에 의해서 행동이 가해졌을 때 기대할 수 있는 미래의 반환값으로서, 정책이 실현되는 시간스텝 t에서의 기댓값이기 때문에 목적함수의 그래디언트를 계산할 때, 에피소드가 끝날 때까지 기다릴 필요가 없다. 

## 4.3 분산을 감소시키기 위한 방법 
분산은 데이터가 평균값으로부터 얼마나 넓게 산재해 있느냐를 나타내는 척도이다. 목적함수 그래디언트의 분산이 크다면 업데이트될 정책 파라미터값이 들쭉날쭉하므로 신경망 학습이 불안정해지고 정책의 불확실성도 커진다. 따라서 목적함수 그래디언트의 분산을 줄이기 위한 방법을 알아본다.  
$$\nabla_\theta J(\theta)=\sum _{t=0} ^T (\int _{(x_t, u_t)} \gamma^t \nabla_\theta \log \pi_\theta(u_t|x_t)Q^{\pi_\theta}(x_t, u_t)p_\theta(x_t,u_t)dx_tdu_t)$$
$Q^{\pi_\theta}(x_t, u_t)$대신 어떠한 파라미터 $b_t$를 넣고 $\nabla_\theta \log \pi_\theta(u_t|x_t)= {{\nabla_\theta \pi_\theta(u_t|x_t)}\over{\pi_\theta(u_t|x_t)}}$ 를 적용하면 다음과 같아진다. 
$$\nabla_\theta J(\theta)=\sum_{t=0}^T(\int_{x_t}[\int_{u_t}\nabla_\theta \pi_\theta(u_t|x_t)b_tdu_t]p_\theta(x_t)dx_t)$$
$b_t$가 상수이거나 행동 $u_t$의 함수가 아니라고 가정하면 위 식에서 대괄호 안의 식은 다음과 같이 된다.
$$\int_{u_t}\nabla_\theta \pi_\theta(u_t|x_t)b_tdu_t=b_t\nabla_\theta\int_{u_t}\pi_\theta(u_t|x_t)du_t = b_t\nabla_\theta(1)=0$$
$$\therefore \nabla_\theta J(\theta)=0$$
목적함수 그래디언트 식의 $Q^{\pi_\theta}(x_t, u_t)$에서 $b_t$를 빼도 기댓값은 변하지 않는다.
$$\nabla_\theta J(\theta)=\sum _{t=0} ^T (E_{x_t \sim d_\theta (x_t),u_t \sim \pi_\theta(u_t|x_t)}[\nabla_\theta \log \pi_\theta(u_t|x_t)(Q^{\pi_\theta}(x_t, u_t)-b_t)])$$
여기서 $b_t$를 베이스라인(baseline)이라 한다. 베이스라인을 도입함으로써 목적함수 그래디언트의 분산을 줄일 수 있다. 일반적으로 상태가치 함수 $V^{\pi_\theta}(x_t)$를 베이스라인으로 사용한다. 상태가치 함수는 행동 $u_t$의 함수가 아니기 때문에 베이스라인으로 사용해도 무방하다.  
$$\nabla_\theta J(\theta)=\sum _{t=0} ^T (E_{x_t \sim d_\theta (x_t),u_t \sim \pi_\theta(u_t|x_t)}[\nabla_\theta \log \pi_\theta(u_t|x_t)(Q^{\pi_\theta}(x_t, u_t)-V^{\pi_\theta}(x_t))])$$
$$\nabla_\theta J(\theta)=\sum _{t=0} ^T (E_{x_t \sim d_\theta (x_t),u_t \sim \pi_\theta(u_t|x_t)}[\nabla_\theta \log \pi_\theta(u_t|x_t)(A^{\pi_\theta}(x_t, u_t))])$$
$A^{\pi_\theta}(x_t, u_t)$는 어드밴티지(advantage)함수라고 한다. 어드밴티지 함수는 상태변수 $x_t$에서 선택된 행동 $u_t$가 평균에 비해 얼마나 좋은지를 평가하는 척도로 해석할 수 있다. 이제 목적함수 그래디언트는 행동가치가 아니라 어드밴티지에 비례한다. 어드밴티지의 정의상 그 값이 행동가치보다 작으므로 그래디언트의 분산이 작아질 것으로 기대할 수 있다. 하지만 여기서 문제는 어드밴티지 값을 모른다는 것이다. 결국 분산을 줄이는 문제는 어드밴티지 함수를 얼마나 정확하게 추정하느냐에 달려 있다.

## 4.4 A2C 알고리즘


$A^{\pi _{\theta }}(x_{t },u_{t } )=Q^{\pi _{\theta }}(x_{t },u_{t } )-V^{\pi _{\theta }}(x_{t })\approx r(x_{t },u_{t } )+\gamma V^{\pi _{\theta }}(x_{t+1 } )-V^{\pi _{\theta }}(x_{t })$

- 어드밴티지가 다음과 같이 바뀌어 상태가치를 정확히 계산하면 된다.

-정책 신경망 = 액터 신경망
-가치 신영망 = 크리틱 신경망

- 가치함수를 근사하기 위해 벨만 방정식 이용

$V^{\pi }=E_{u_{t}\sim \pi (u_{t}|x_{t})}[r(x_{t },u_{t } )+E_{x_{t+1}\sim p(x_{t+1 }|x_{t},u_{t } )}\left [ \gamma V^{\pi }(x_{t+1})\right]]$

$V_{\phi }(x_{t})\approx r(x_{t },u_{t } )+\gamma V_{\phi}(x_{t+1})$

시간차 타깃 : $y_{i}=r(x_{i},u_{i})+\gamma V_{\phi }(x_{i+1})$

- 손실함수가 최소가 되는 근사 가치함수

$Loss_{critic}(\phi )=\frac{1}{2}\sum_{i}^{}\left \| r(x_{i},u_{i})+\gamma V_{\phi }(x_{i+1})-V_{\phi }(x_{i}) \right \|^{2}$
-어드밴티지 근사

$A_{\phi}(x_{i},u_{i})\approx r(x_{i},u_{i})+\gamma V_{\phi }(x_{i+1})-V_{\phi }(x_{i})$

- 액터 신경망의 손실함수

$Loss_{actor}(\theta )\approx -\sum_{i}^{}(log\pi _{\theta }(u_{i}|x_{i})A_{\phi }(x_{i},u_{i}))$

$\theta \leftarrow \theta -\alpha \triangledown _{\theta }\sum_{i}^{}(log\pi _{\theta }(u_{i}|x_{i})A_{\phi }(x_{i},u_{i}))$

어드밴티지는 $\theta$의 함수가 아니기 때문에 그래디언트 안에 포함.

손실함수를 최소화하도록 파라미터가 엡데이트되고, 목적함수는 최대로 해야하기 때문에 마이너스 부호.

- 액터-크리틱 알고리즘

  : N개의 샘플->정책 업데이트->반복
  1. 크리틱과 액터 신경망의 파라미터 $\phi,\theta$ 초기화
  2.  반복
    1. N시간스텝 동안 아래의 과정 반복
      1. 정책으로 행동을 확률적으로 선택
      2. 행동의 보상과 다음 상태변수를 측정
      3. 샘플 저장
    2. 시간차 타깃 계산
    3. 크리틱 신경망의 손실함수 계산 

    $L=\frac{1}{2}\sum_{i}^{}(y_{i}-V_{\phi }(x_{i}))^{2}$
    4. 어드밴티지 계산
    
    $A_{\phi}(x_{i},u_{i})= r(x_{i},u_{i})+\gamma V_{\phi }(x_{i+1})-V_{\phi }(x_{i}), i=1,\cdots ,N$

    5. 크리틱 신경망 업데이트

    $\phi\leftarrow \phi +\alpha_{critic} \sum_{i}^{}\left [ (y_{i}-V_\phi (x_{i}))\triangledown _{\phi }V_{\phi }(x_i) \right ]$

    6. 액터 신경망 업데이트

    $\theta \leftarrow \theta+\alpha_{actor} \triangledown _{\theta }\sum_{i}^{}(log\pi _{\theta }(u_{i}|x_{i})A_{\phi }(x_{i},u_{i}))$


- 온라인 액터-크리틱 알고리즘
  : 한 개의 샘플->신경망 업데이트-> 반복
  1. 크리틱과 액터 신경망의 파라미터 $\phi,\theta$ 초기화
  2. 반복
    1. 정책으로 행동을 확률적으로 선택
    2. 행동의 보상과 다음 상태변수 측정
    3. 시간차 오차 계산
    
    $\rho _{t}= r(x_{t},u_{t})+\gamma V_{\phi }(x_{t+1})-V_{\phi }(x_{t})$

    4. 크리틱 신경망 업데이트
    
    $\phi\leftarrow \phi +\alpha_{critic}\rho _{t} \triangledown _{\phi }V_{\phi }(x_{t}) $

    5. 액터 신경망 업데이트

    $\theta \leftarrow \theta+\alpha_{actor} \triangledown _{\theta }(\rho_tlog\pi _{\theta }(u_{t}|x_{t}))$

- 이산 공간

  액터 신경망의 출력층 뉴런의 개수 = 이산화된 행동변수가 가질 수 있는 값의 개수

  액터 신경망의 출력 = 행동변수가 가질 수 있는 값의 확률

  ex) 행동변수 $x=\left [ x_1\cdots x_j\cdots x_n \right ]^T$의 차원이 n, 행동변수가 가질 수 있는 값 $\mu =\left [ \mu _1 \mu _2\cdots \mu _m \right ]^T$의 개수가 m개이면 **신경망의 출력의 뉴런 개수는 nm개**

- 연속 공간

  각 행동변수 1개당 가질 수 있는 값 = 무한개 -> 무한개의 뉴런 개수가 필요

=> 구현 가능한 액터 신경망을 만들기 위해서 신경망의 출력을 특정한 구조를 갖는 확률밀도함수로 고정해야한다.
